# Type Hints

## Динамично vs. статично типизиране. Round 2

Динамичното типизиране позволява гъвкавост, но това идва и със своите недостатъци от гледна точка на поддръжката и четимостта на кода.

Например, ако имаме следната функция:

In [1]:
def validate_serial_number(serial_number):
    pass  # definition is irrelevant for this example

Ако нямаме поглед над имплементацията на функцията (или пък имаме, но не е тривиално да се прецени точно имплементацията с какви типове борави), то тогава как да разберем `serial_number` дали да ѝ го подадем като `int` или `str`? Или пък може да е `list` от `int`-ове дори?

Също така, от името на функцията не става много ясно тя какво връща - дали не връща нищо, а само изпълнява проверки и странични ефекти, или пък проверява валидността на номера и връща `bool`, или пък връща някакъв тип за грешка при неуспех, а `None` при успех?

## Type Hints and `mypy`

От Python 3.5 насам можем да пишем "подсказки" за очаквания тип (или очакваните типове, в случай че са повече от един).

В случая функцията можем да я анотираме по следния начин:

In [2]:
def validate_serial_number(serial_number: str) -> bool:
    pass  # ...

Трябва да се отбележи, че тези анотации са само hints ("подсказки" за програмиста), т.е. не получаваме поведението на статично типизираните езици, тъй като интерпретатора не следи за спазването на анотациите:

In [4]:
def add(a: int, b: int) -> int:
    return a + b

print("Passing `int`: ", add(1, 2))
print("Passing `str`: ", add("1", "2"))

Passing `int`:  3
Passing `str`:  12


Това въпросно следене може да стане чрез т.нар. "type checkers". Най-използваният е [mypy](http://mypy-lang.org/).

В PyCharm би трябвало по подразбиране да е включено, докато във VS Code може да се включи от настройките на [Python extension-a](https://marketplace.visualstudio.com/items?itemName=ms-python.python):

![mypy in VS Code](assets/mypy-vscode.png)

При включването на настройката за пръв път би трябвало да се покаже долу вдясно диалогов попъп, от който да може да се инсталира `mypy` автоматично.

След включването, всяка некоректност ще се показва като грешка във VSCode по подобен начин:

![mypy in action](assets/mypy-in-action.png)

## Синтаксис и особености на анотациите

Анотациите са добавени в Python 3.0 и оригинално са нямали конкретна семантика или общоприета употреба. Понеже са интуитивно удобни за указване на типове, [PEP484](https://peps.python.org/pep-0484/) и [PEP526](https://www.python.org/dev/peps/pep-0526/) предлагат и стандартизират точно тази им употреба.

Анотации могат да се добавят към променливи и функции. Те отиват в `__annotations__` списъка на модула или на функцията, респективно. Това означава, че могат и runtime да бъдат достъпни:

In [8]:
variable: int = 42

print(f"{__annotations__ = }")  # module.__annotations__ keeps the annotations of all variables in the module


def function(f: float, b: bool = True) -> int:
    return int(f) if b else 0  # this is the most meaningless function ever

print(f"{function.__annotations__ = }")  # functions have that dunder as well

__annotations__ = {'variable': <class 'int'>}
function.__annotations__ = {'f': <class 'float'>, 'b': <class 'bool'>, 'return': <class 'int'>}


*Забележка*: По общоприета питонска конвенция за стил, интервали около `=` на именованите аргументи се слагат ако има анотация, но се пропускат ако няма:

ОК:
```python
def foo(a: int, b: int = 0) -> int:
    return a + b

def foo(a, b=0):
    return a + b
```

Not quite OK:

```python
def foo(a: int, b: int=0) -> int:
    return a + b

def foo(a, b = 0):
    return a + b
```

*Забележка 2*: Възможна е анотация на имe без да му бъде присвоявана стойност. Опитът за достъп обаче преди да му бъде присвоена такава впоследствие ще доведе до грешка, понеже още не е дефинирана:

In [13]:
name: str

print(name)

NameError: name 'name' is not defined

## Анотиране на различните вградени типове

Както видяхме, използваме самият тип за да го анотираме, т.е. `bool`, `int`, `float`, `complex`, `str`, `bytes`, `None` и т.н. са валидни в анотации. (да, `None` освен стойност, е и тип сам по себе си)

In [ ]:
def i_wanna_print(something: str, terminator: str = "\n") -> None:
    print(something, terminator=terminator)

def extract_nums_from_input_row(row: str) -> list:
    return list(map(int, row.split()))

Във функцията `extract_nums_from_input_row` в горният пример обаче не знаем всичко за return типа. Знаем, че е `list`, но лист от какво?

Проблемът с такива generic типове като `list`, `tuple`, `dict`, `set` и т.н. решаваме като укажем типа на елементите в тях в квадратни скоби. От Python 3.9 насам можем директно да ползваме builtin типовете, но за версии 3.5 до 3.8 вкл. трябва да импортнем подходящите класове от `typing` модула:

In [11]:
# Python 3.5 to 3.8

from typing import List, Tuple, Dict

def extract_nums_from_input_row(row: str) -> List[int]:
    return list(map(int, row.split()))

def multiply(a: Tuple[int, int, int], b: Tuple[int, int, int]) -> Tuple[int, int, int]:
    return sum(x * y for x, y in zip(a, b))

def bind_names_to_ages(names: List[str], ages: List[int]) -> Dict[str, int]:
    return dict(zip(names, ages))

In [12]:
# Python 3.9+

def extract_nums_from_input_row(row: str) -> list[int]:
    return list(map(int, row.split()))

def multiply(a: tuple[int, int, int], b: tuple[int, int, int]) -> tuple[int, int, int]:
    return sum(x * y for x, y in zip(a, b))

def bind_names_to_ages(names: list[str], ages: list[int]) -> dict[str, int]:
    return dict(zip(names, ages))

Ако искаме `tuple` да има точно 2 елемента от тип `int` например, можем да го анотираме като `Tuple[int, int]`. Ако искаме да има произволен брой елементи от тип `int`, можем да го анотираме като `Tuple[int, ...]`.

In [31]:
def calculate_polynomial(coefficients: tuple[float, ...], x: float) -> float:
    return sum(k * (x ** n) for n, k in enumerate(reversed(coefficients)))

При указване на `*args` и/или `**kwargs` е нужно да укажем само типа на съответните елементи, без `tuple` или `dict`:

In [32]:
def calculate_polynomial(*coefficients: float, x: float) -> float:
    return sum(k * (x ** n) for n, k in enumerate(reversed(coefficients)))

За удобство можем да си създваме alias-и за различни типове, които да ги използваме по-късно:

In [28]:
Vector3D = tuple[float, float, float]

def multiply(a: Vector3D, b: Vector3D) -> Vector3D:
    return sum(x * y for x, y in zip(a, b))

В случай, че искаме да анотираме функция, която например подаваме като параметър, ползвамe `Callable`:

In [38]:
from typing import Callable

def bubble_sort(arr: list[int], comparator: Callable[[int, int], bool]) -> list[int]:
    arr = arr.copy()
    for _ in range(len(arr)):
        for j in range(len(arr) - 1):
            if not comparator(arr[j], arr[j + 1]):
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr

bubble_sort([1, 2, 3], lambda x, y: x > y)

[3, 2, 1]

### `Union` и `Optional`

Понякога можем да поддържаме по няколко възможни типа. Целта на `Union` е да "обедини" тези типове в един. Например, ако искаме да можем да подаваме и `int` и `str` като аргументи на функция, можем да го анотираме като `Union[int, str]`  или  `Union[str, int]`:

In [39]:
from typing import Union

def is_serial_number_valid(serial_number: Union[str, int]) -> bool:
    serial = str(serial_number)
    return (
        len(serial) == 10 
        and serial.isdigit() 
        and serial[-1] == sum(map(int, serial[:-1])) % 10
    )

От Python 3.10 насам можем да използваме оператора `|` вместо `Union`:

In [47]:
def is_serial_number_valid(serial_number: str | int) -> bool:
    serial = str(serial_number)
    return (
        len(serial) == 10 
        and serial.isdigit() 
        and serial[-1] == sum(map(int, serial[:-1])) % 10
    )

def calculate_polynomial(*coefficients: int | float | complex, x: int | float | complex) -> complex:
    return complex(sum(k * (x ** n) for n, k in enumerate(reversed(coefficients))))

calculate_polynomial(-1, 5j, 6.9, 0, 94, x=42)

(-3099430.4+370440j)

Много често се налага да имаме стойност по подразбиране `None` вместо такава на даден тип или пък да връщаме `None` вместо стойност от даден тип. Това е и идеята на `Optional`, който е тип на практика еквивалентен на `Union` с `None`:

In [49]:
import hashlib

from typing import Optional

def hash_password(password: str, salt: Optional[str] = None) -> bytes:
    hash = hashlib.sha256(password.encode())
    if salt is not None:
        hash.update(salt.encode())
    return hash.digest()

In [50]:
# Equivalent:

import hashlib

def hash_password(password: str, salt: str | None = None) -> bytes:
    hash = hashlib.sha256(password.encode())
    if salt is not None:
        hash.update(salt.encode())
    return hash.digest()

### `Any`

Ако искаме даден тип да е произволен, можем да го анотираме с `Any`. Type-checker-ите няма да хвърлят грешка, който и тип да подаваме като стойност на нещо, което се очаква да е от тип `Any`.

In [ ]:
from typing import Any

def i_wanna_print(something: Any, terminator: str = "\n") -> None:
    print(something, terminator=terminator)

def play_audio(filename: str, options: dict[str, Any]) -> None:
    if options["BITRATE"] == 320:
        print("Playing in high quality")
    
    if options["LOOP"]:
        print("Playing in loop")

    #...

### Generics

В някои случаи обаче ако използваме `Any` директно всъщност заличаваме информация, която ни е нужна после. 

Например нека разгледаме следната функция:

In [56]:
import random
from typing import Any

def shuffled(l: list[Any]) -> list[Any]:
    return random.sample(l, len(l))

 Ако я използваме обаче в следния контекст:

In [72]:

suits = "♠♥♦♣"
ranks = "AKQJT98765432"

brand_new_deck = [f"{rank}{suit}" for suit in suits for rank in ranks] # -> list[str]

shuffled_deck = shuffled(brand_new_deck) # -> list[Any] !!!

# Сега `shuffled_deck` за type-checker-ите е list[Any] вместо list[str]
# т.е. изгубихме типовата информация, която би трябвало да имаме.
# Когато се опитаме да направим нещо спечицично за `str`
# ще ни го подчертаят като невъзможно или неопределено.

from collections import Counter
suits_dealt = Counter(card[1] for card in shuffled_deck[::4]) # card[1] ще ни го дава като опит за индексиране на `Any`, а не `str`
print(suits_dealt)

Counter({'♠': 5, '♥': 4, '♦': 2, '♣': 2})


За да се справим с този проблем трябва да обявим такива типови параметри като generics чрез `TypeVar`:

In [74]:
from typing import TypeVar

T = TypeVar("T")
def shuffled(l: list[T]) -> list[T]:
    return random.sample(l, len(l))

С примерът горе статичния type-checker ще знае, че каквито елементи има листът, който е подаден като аргумент на `shuffled`, такива и ще бъдат елементите на върнатия лист.

### Собствени типове

Нашите собствени класове също могат да се използват като типове:

In [76]:
class Person:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    def __repr__(self) -> str:
        return f"Person(name={self.name}, age={self.age})"

    def to_dict(self) -> dict[str, Any]:
        return {"name": self.name, "age": self.age}


def get_people() -> list[Person]:
    return [Person("John", 42), Person("Jane", 37)]

Тук особеното е, че в дефиницията на методи няма нужда да анотираме `self` - той винаги ще е от типа на класа.

Когато обаче имаме метод, който приема или връща обект от типа на класа, към който е, съществуват особености за различните версии на езика. В Python 3.10 не е грешка да се използва името на класа още в дефиницията му, докато в по-старите версии - е. От Python 3.7+ може да се импортне `from __future__ import annotations`, за да работи това, докато за по-стари версии решението е просто да се напише името на класа като стринг в анотацията.

In [78]:
class Person:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    def __repr__(self) -> str:
        return f"Person(name={self.name}, age={self.age})"

    def to_dict(self) -> dict[str, Any]:
        return {"name": self.name, "age": self.age}

    # Python 3.10+ way
    # or for Python 3.7-3.9 you also have to add `from __future__ import annotations`
    @classmethod
    def from_dict(cls, data: dict[str, Any]) -> Person:
        return cls(**data)

    # Python <3.7 way
    @classmethod
    def from_dict(cls, data: dict[str, Any]) -> "Person":
        return cls(**data)

Можем да си декларираме собствени типове, които да са съставени от други типове. Това се прави с `NewType`:

In [86]:
from typing import NewType, Optional

PersonId = NewType("PersonId", int)

class Person:
    def __init__(self, id: PersonId, name: str, age: int) -> None:
        self.id = id
        self.name = name
        self.age = age

    def __repr__(self) -> str:
        return f"Person(id={self.id}, name={self.name}, age={self.age})"

    def to_dict(self) -> dict[str, Any]:
        return vars(self)

    @classmethod
    def from_dict(cls, data: dict[str, Any]) -> Person:
        return cls(**data)

def get_person_by_id(id: PersonId, database: list[Person]) -> Optional[Person]:
    return next((person for person in database if person.id == id), None)  # `next` has an optional second parameter - default value

database = [
    Person(PersonId(37), "Lana Xes", 18),
    Person(PersonId(69), "Axl Rose", 60),
]

print(get_person_by_id(PersonId(37), database))
print(get_person_by_id(PersonId(666), database))

Person(id=37, name=Lana Xes, age=18)
None
